In [14]:
import numpy as np
import pandas as pd
import os

from sklearn.metrics import classification_report
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tqdm import tqdm

print(f"TensorFlow has access to the following devices:\n{tf.config.list_physical_devices()}")


TensorFlow has access to the following devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
class_names = ['NORMAL', 'COVID19', 'PNEUMONIA', 'TUBERCULOSIS']
class_names_label = {class_name: i for i, class_name in enumerate(class_names)}
nb_classes = len(class_names)

print(class_names_label)

IMAGE_SIZE = (250,250)


{'NORMAL': 0, 'COVID19': 1, 'PNEUMONIA': 2, 'TUBERCULOSIS': 3}


In [3]:
#Loading Data
def load_data():
    directory = r"multi-disease-data"
    category = ["train", "test"]
    
    output = []
    
    for cat in category:
        path = os.path.join(directory, cat)
        print(path)
        images = []
        labels = []
        
        print("Loading {}".format(cat))
        
        for folder in os.listdir(path):
            if not folder.startswith('.'):
                
                label = class_names_label[folder]

                for file in os.listdir(os.path.join(path,folder)):
                    img_path = os.path.join(os.path.join(path,folder), file)

                    image = cv2.imread(img_path)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image = cv2.resize(image, IMAGE_SIZE)

                    images.append(image)
                    labels.append(label)
                
        images = np.array(images, dtype= 'float32')
        labels = np.array(labels, dtype = 'int32')
        
        output.append((images,labels))
        
    return output
                
                

In [4]:
(train_images, train_labels), (test_images, test_labels) = load_data()


multi-disease-data/train
Loading train
multi-disease-data/test
Loading test


In [ ]:
train_images, train_labels = shuffle(train_images,train_labels,random_state=25)

In [5]:
def display_examples(class_name, images, labels):
    figsize = (150,150)
    fig = plt.figure(figsize = figsize)
    fig.suptitle("Example images from dataset", fontsize = 150)
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        image = cv2.resize(images[i],figsize)
        plt.imshow(image.astype(np.uint8))#, cmap=plt.cm.binary
        plt.xlabel(class_names[labels[i]],fontsize = 100)
    plt.show()

In [6]:
display_examples(class_names, train_images, train_labels)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
import tensorflow as tf
import tensorflow_hub as hub

In [16]:
#MODEL 

model = Sequential([
    Conv2D(32,(3,3), activation = 'relu', input_shape = (250,250, 3)),
    MaxPooling2D(2,2),
    
    Dropout(0.25),
    
    Conv2D(64,(3,3),  activation = 'relu'),
    MaxPooling2D(2,2),
    
    Dropout(0.25),
    Conv2D(128,(3,3),  activation = 'relu'),
    Dropout(0.4),
   
    
    Flatten(),
    Dense(128, activation ='relu'),
    Dropout(0.3),
    Dense(nb_classes, activation='softmax')
    
])



In [17]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 248, 248, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 124, 124, 32)     0         
 2D)                                                             
                                                                 
 dropout_12 (Dropout)        (None, 124, 124, 32)      0         
                                                                 
 conv2d_10 (Conv2D)          (None, 122, 122, 64)      18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 61, 61, 64)       0         
 2D)                                                             
                                                                 
 dropout_13 (Dropout)        (None, 61, 61, 64)       

In [18]:
model.compile(optimizer = 'adam', loss= 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, batch_size = 128, epochs=4, validation_split = 0.2) 

Epoch 1/4


2022-08-07 10:48:44.503749: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


13/40 [========>.....................] - ETA: 53s - loss: 1.3801 - accuracy: 0.7037

In [ ]:
def plot_accuracy_loss(history):
    fig = plt.figure(figsize=(10,5))
    
    #accuracy funct
    plt.subplot(221)
    plt.plot(history.history['accuracy'],'bo--', label = "acc")
    plt.plot(history.history['val_accuracy'], 'ro--', label = "val_acc")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()
    
    #loss funct
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")
    plt.legend()
    
    
    plt.show()
    
    

In [ ]:
plot_accuracy_loss(history)